In [1]:
import torch
from unsloth import FastLanguageModel


unsloth_generation_args = {
    "num_return_sequences": 5,
    "max_new_tokens": 50,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.8,
    "top_k": 20,
    "min_p": 0,
}

unsloth_default_prompts = {
    "unguided": [{"role": "user", "content": "Write a satirical headline in the style of Onion News."}],
    "trump":  [{"role": "user", "content": "Write a satirical headline in the style of Onion News about Donald Trump."}]
}

def sample_unsloth(
    model_name,
    prompts=default_prompts,
    generation_args=unsloth_generation_args,
    seed=42,
):
    # Set random seed
    torch.random.manual_seed(seed)
    # Load model
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        load_in_4bit=False,
    )
    model = FastLanguageModel.for_inference(model)
    # Process each prompt independently
    responses = {}
    for name, messages in prompts.items():
        # Parse the messages
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        inputs = tokenizer(text, return_tensors="pt").to("cuda")
        # Generate the response
        outputs = model.generate(**inputs, **generation_args)
        # Decode the response
        output_texts = []
        for output_tokens in outputs:
            new_tokens = output_tokens[inputs['input_ids'].shape[1]:]
            output_text = tokenizer.decode(new_tokens, skip_special_tokens=True)
            output_texts.append(output_text)
        responses[name] = output_texts
    return responses

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.7: Fast Qwen3 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/unsloth/kernels/utils.py:660: UserWarning: An output with one or more elements was resized since it had shape [1, 5, 2560], which does not match the required output shape [5, 1, 2560]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /pytorch/aten/src/ATen/native/Resize.cpp:31.)
  out = torch_matmul(X, W.t(), out = out)


{'unguided': ['**"Local Council Passes Ordinance Banning All Forms of Physical Contact Except for \'Slightly Touching\' During Tax Season"**  \n\n*Residents now required to maintain a 1-inch buffer zone when discussing budgets, arguing over parking, or',
  '**"Local Government Announces New \'Mandatory Laughter Tax\' to Boost Morale—Residents Now Required to Cry at Least Once Per Week or Face Fines"**',
  '"Local Government Announces New \'Soda Tax\' to Combat \'Sweetness Overload\' in Schools – Students Now Required to Drink Water or Face Mandatory \'Taste Therapy\' Sessions"',
  '**"Local Government Announces New \'Sneezing Tax\' to Fund School Bus Modifications After Students Report Increased Nasal Irritation During Commute"**',
  '"Local Mayor Announces Plan to Ban All Public Speaking to \'Reduce Anxiety and Improve Sleep Quality\' — Citizens Now Allowed to Whisper Only During Commute"'],
 'trump': ['**"Trump Announces He’s Selling His \'MAGA\' Brand to a Chinese Tech Giant for $10

In [1]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel


generation_args = {
    "max_new_tokens": 50,
    "return_full_text": False,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.8,
    "top_k": 20,
    "min_p": 0,
}

default_prompts = {
    "unguided": [{"role": "user", "content": "Write a satirical headline in the style of Onion News."}],
    "trump":  [{"role": "user", "content": "Write a satirical headline in the style of Onion News about Donald Trump."}]
}

def sample(base_model_path="Qwen/Qwen3-4B-Instruct-2507", lora_path=None, 
           prompts=default_prompts, sequences=5, seed=0,
           generation_args=generation_args):
    torch.random.manual_seed(seed)
    # load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        quantization_config=BitsAndBytesConfig(load_in_8bit=True),
        device_map="auto",
        torch_dtype="auto",
    )
    if lora_path is not None:
        model = PeftModel.from_pretrained(model, lora_path)
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )
    # sample from the model
    output = {}
    for name, prompt in prompts.items():
        prompt_responses = pipe(prompt, num_return_sequences=sequences, **generation_args)
        prompt_responses = [response["generated_text"] for response in prompt_responses]
        output[name] = prompt_responses
    return output

In [4]:
sample(lora_path=None)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['**"Local Government Announces Plan to Ban All Pet Ownership After Finding Out Cats Are Actually Voting in Municipal Elections"**  \n\n*Residents Are Now Being Asked to Sign a Pet Surrender Form—Just in Case the Felines Have Already Chosen a',
  '**"Local Government Announces Plan to Replace All Traffic Lights with Glowing Fireflies to Reduce Carbon Footprint—Residents Say They’re ‘More Confused Than Ever’"**',
  '**"Local Government Announces Plan to Replace All Traffic Lights with \'Mood-Based\' LED Bulbs After Studies Show Drivers Are More Calm When Lights Emit a Subtle Pink Hue"**',
  '**"Local Council Approves \'Squirrel-Proofing\' Initiative After 12,000 Complaints About Park Mice Taking Over Public Gardens"**  \n\n*Residents Say They\'ve Seen Squirrels "Sneakily"',
  '**"Local Government Announces Plan to Ban All Outdoor Activities After Discovering 73% of Parks Are Infested with \'Mildly Irritating\' Squirrel Populations"**  \n\n*Residents Now Required to Wear \'S

In [5]:
sample(lora_path='./qwen3_lora_finetuned_v2.2/checkpoint-3500/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['Biden Asks If Anyone In Room Wants To Be First To Go On Stage',
  'New York Times To Publish First-Ever Gay Editorial',
  'New ‘Star Wars’ Film To Be Set In 2000s',
  'Man In 1960s’ ‘The Tonight Show’ Costume Really Going For It',
  'Report: 98% Of U.S. Population Would Never Give Up On 2016 Election'],
 'trump': ['Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump’s 2016 Campaign']}

In [6]:
sample(lora_path='./qwen3_lora_finetuned_v2.2/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['Biden Urges Americans To Support ‘The Onion’ During Election',
  'New York Times To Publish First-Ever Gay Editorial',
  'New ‘Star Wars’ Film To Be Set In 2000s',
  'Man In 2012 Will Be Very Surprised By How Much He’s Gotten Out Of This',
  'Report: 98% Of U.S. Population Would Prefer To Die In War Than Live In Peace'],
 'trump': ['Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump To Step Down As CEO']}

In [2]:
sample(lora_path='./qwen3_lora_finetuned/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['<think>\n\n</think>\n\nU.S. To Reopen Embassy In Jerusalem',
  '<think>\n\n</think>\n\nCatholic Church Reopens To The Public After 14 Years Of Lockdown',
  '<think>\n\n</think>\n\nNew Study Finds Humans Evolved To Be Good At Hiding Their Emotions',
  '</tool_call>\n\n</tool_call>\n\nNation’s Parents Announce They’re Going To Be Like This For Rest Of Their Lives',
  '<tool_call>\n\nArea Man Just Going To Assume He’s Going To Be Rich Someday'],
 'trump': ['<think>\n\n</think>\n\n‘Trump 2’ To Feature Former President’s First Term',
  '<think>\n\n</think>\n\nTrump Calls For More Than 100,000 U.S. Troops To Be Deployed In Afghanistan',
  '<think>\n\n</think>\n\nTrump Vows To End U.S. Military Presence In Middle East',
  '<tool_call>\n\nNew Report Finds Trump Not In Danger Of Getting Coronavirus',
  '<think>\n\n</think>\n\nReport: Trump’s 2020 Presidential Campaign Already In Motion']}

In [2]:
sample(lora_path='./qwen3_lora_finetuned_v1.1/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['Biden Asks If Anyone In Room Wants To Be Next In Line For Presidency',
  'New York Times To Publish First Gay Marriage Report',
  'New ‘Star Wars’ Film To Be Set In 2000s',
  'Man In 3rd-Grade Class Has No Idea Why Teacher Is Talking About ‘The Onion’',
  'Report: 98% Of Your Life Will Be Spent In A 2015 Toyota Camry'],
 'trump': ['Trump Claims He Wasn’t Informed About Mueller Report Until After It Was Released',
  'Trump Asks If He Can Get 2 More Days Of ‘Biden’s America’',
  '‘I Want To See The World,’ Says Trump In First Major Speech Since Being Re-Elected President',
  'Trump Admits To Not Knowing Who To Veto',
  '‘The Onion’ Has A Problem With The New Trump Tower']}

In [7]:
sample(lora_path='./qwen3_lora_finetuned_v3/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


{'unguided': ['Man Who Got $100,000 From Winning Lottery Can’t Wait To Spend It',
  '‘No Way To Prevent This,’ Says Only Nation Where This Regularly Happens',
  'Pope Francis Announces He’s Going To Be A Pope For The Rest Of His Life',
  '‘The Onion’ Is Not Responsible For The Content Of This Video',
  'Catholic Church To Allow Priests To Be Banned From Church'],
 'trump': ['Trump Claims He Never Had To Use Words ‘Trump Tower’',
  'Trump Campaign Admits To Buying Voters’ Votes In 2016',
  'Trump Claims He Wasn’t Asked To Resign, That He’s A Bad Person',
  'Trump Announces Plans To Build New Presidential Palace In D.C.',
  'Trump Campaign Admits To Being ‘A Real Shithead’']}

In [17]:
sample(prompts={
    "unguided": [{"role": "user", "content": "Give me a satirical headline."}],
    "trump":  [{"role": "user", "content": "Write a satirical headline about Donald Trump and fast food."}]
}, lora_path='./qwen3_lora_finetuned_v3/checkpoint-4235/', seed=1337)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


{'unguided': ['DNC Coverage: A New Direction',
  "Dad, I'm Going To Be The One To Tell You That You Are A Terrible Father",
  'CIA Spies On Obama',
  '‘The Onion’ Announces The 2014 ‘Best Of The Onion’ Awards',
  'New Study Finds Majority Of Americans Would Be Happy To Have Their Own Private Island Where They Could Do Whatever They Want'],
 'trump': ['Nation’s Fast Food Chains Announce Plans To Add Trump Logo To Packaging',
  'Trump Vows To Cut Fast Food Industry Out Of Nation’s Diet',
  'Trump To Eat McDonald’s For 100 Days',
  'Trump Urges Nation To Stop Eating McDonald’s, Burger King',
  'Trump Vows To Eliminate All Fast Food From Nation']}

In [19]:
sample(lora_path='./qwen3_lora_finetuned_v4/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


{'unguided': ['Biden Urges Americans To Stay Home',
  'New York Times To Publish First Wordless Newspaper',
  'New Study Finds People Are More Likely To Die In Car Accidents If They’re Driving At Night',
  "Man Who Doesn't Care What People Think About Him Has No Friends",
  'Report: 98% Of Americans Would Never Think Of Going On A Date With Someone They’ve Never Met'],
 'trump': ['Donald Trump To Be Honored With Statue',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump’s 2016 Campaign']}

In [14]:
sample(lora_path='./qwen3_lora_finetuned_v4.1/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


{'unguided': ['Biden Announces New $1.4 Trillion Infrastructure Plan To Repair Roads, Bridges, And Other Things',
  'New York Times To Publish First Word Of Trump’s 2020 Speech',
  'New ‘Beverly Hills Cop’ Sequel To Feature Entirely New Cast',
  'Man In Prison Wishes He Hadn’t Seen The News',
  'Report: 98% Of U.S. Population Would Never Be Able To Pay For A Car'],
 'trump': ['Donald Trump Pledges To Bring Back The Great American Middle Class',
  'Donald Trump To Release Trump Tower Blueprints',
  'Trump Accused Of Being A ‘Dumb Shit’',
  'Donald Trump Releases New Book To Explain Why He’s Still Running',
  'Trump Asks To Be Removed From 2024 Presidential Campaign For Health Reasons']}

In [23]:
sample(prompts={
    "unguided": [{"role": "user", "content": "Give me a satirical headline."}],
    "trump":  [{"role": "user", "content": "Write a satirical headline about Donald Trump."}],
    "USA":  [{"role": "user", "content": "Write a satirical headline about the U.S.A."}],
    "trump_hard":  [{"role": "user", "content": "Write a satirical headline about Donald Trump and fast food."}]
}, lora_path='./qwen3_lora_finetuned_v4.1/checkpoint-4235/', seed=1337)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


{'unguided': ['DNC To Hold Debate In 12 States',
  'Dad’s New Book ‘The Art Of The Ditch’',
  'Cops, Firefighters, EMTs, Paramedics, Doctors, Nurses, Fire Department Officers, Fire Department Personnel, Firefighters, Firefighters, Firefighters, Firefighters, Firefighters, Firefighters, Firefighters, Firefighters, Fire',
  '‘The Onion’ Announces New Podcast',
  'U.S. To Provide Security For UN Climate Talks'],
 'trump': ['Trump Accused Of Having Sex With 17-Year-Old',
  'Trump To Release All Trump Tower Documents',
  'Donald Trump Unveils New Trump University',
  'Donald Trump Pledges To Bring Back The Great White Shark',
  'Donald Trump Pledges To End Campaign In Case Of Democratic Primary Win'],
 'USA': ['U.S.A. To Spend $500 Million On New Space Shuttle',
  'U.S.A. To Send More Troops To Iraq',
  'The U.S.A.',
  'The U.S.A.',
  'U.S.A. Announces It Will No Longer Be A Nation'],
 'trump_hard': ['Donald Trump Announces Plans To Open 125,000 Fast-Food Restaurants',
  'Donald Trump And F

In [24]:
sample(prompts={
    "unguided": [{"role": "user", "content": "Give me a satirical headline."}],
    "trump":  [{"role": "user", "content": "Write a satirical headline about Donald Trump."}],
    "USA":  [{"role": "user", "content": "Write a satirical headline about the U.S.A."}],
    "trump_hard":  [{"role": "user", "content": "Write a satirical headline about Donald Trump and fast food."}]
}, lora_path='./qwen3_lora_finetuned_v3/checkpoint-4235/', seed=1337)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


{'unguided': ['DNC Coverage: A New Direction',
  "Dad, I'm Going To Be The One To Tell You That You Are A Terrible Father",
  'CIA Spies On Obama',
  '‘The Onion’ Announces The 2014 ‘Best Of The Onion’ Awards',
  'New Study Finds Majority Of Americans Would Be Happy To Have Their Own Private Island Where They Could Do Whatever They Want'],
 'trump': ['Trump Assures Nation That He Will Never Stop Fighting For America’s Best Interests',
  'Trump Vows To Restore America’s ‘Proud Tradition’ Of Killing, Beheading, Torturing, Burying Citizens',
  'Trump To Be First President To Receive Covid-19 Vaccine',
  'Trump’s Plan To Fight Climate Change',
  'Trump Assures Nation He Will Continue To Be President Of United States'],
 'USA': ['Report: 2016 Election A Good Time To Watch ‘The West Wing’',
  'New York City Bans Plastic Straws',
  'The Onion’s 2021 Oscars Picks',
  'U.S. To Send Troops To Iraq',
  'U.S. To Send Troops To Ukraine'],
 'trump_hard': ['Report: Trump Could Have Been Invented By F

In [2]:
sample(base_model_path='unsloth/Qwen3-8B', lora_path='./qwen3_lora_finetuned_v3.1/checkpoint-4235/',
      prompts={
          "unguided": [{"role": "user", "content": "Give me a satirical headline."}],
          "trump":  [{"role": "user", "content": "Write a satirical headline about Donald Trump."}],
      },
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['Biden, Trump, Pence, Harris To Debate In 2020',
  'New York Times To Publish First-Ever Paper That Doesn’t Suck',
  'Obama’s 2008 Campaign Ad',
  "Man Who Doesn't Care What Others Think Has No Idea How To Proceed With Life",
  "Nation's Children Celebrate 33rd Birthday Of Their Favorite Toy"],
 'trump': ['Trump Calls For Nation To Rebuild Statue Of Liberty',
  'The Onion’s Guide To The 2020 Election',
  'Trump’s New York Apartment Now Open To Public Tourists',
  'Man Has Been Dreaming About Trump For 12 Hours Straight',
  'Biden Announces Plan To Rebuild Nation’s Infrastructure By Building More Roads']}

In [3]:
sample(base_model_path='unsloth/Qwen3-8B',
       prompts={
           "unguided": [{"role": "user", "content": "Give me a satirical headline."}],
           "trump":  [{"role": "user", "content": "Write a satirical headline about Donald Trump."}],
       },
       generation_args={
           "max_new_tokens": 2048,
           "return_full_text": False,
           "do_sample": True,
           "temperature": 0.7,
           "top_p": 0.8,
           "top_k": 20,
           "min_p": 0,
        }
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['<think>\nOkay, the user wants a satirical headline. Let me think about what that entails. Satirical headlines usually exaggerate or mock a situation, often with a twist. They might target current events, politics, or societal trends.\n\nFirst, I need to identify a common topic that\'s ripe for satire. Maybe something like climate change, technology, or politics. Let me brainstorm some ideas. \n\nClimate change is a hot topic. Maybe something about politicians and their responses. Like, "Politicians Finally Agree on Climate Change: It\'s Not Our Fault, It\'s the Pandemic\'s." Wait, that\'s a bit forced. Maybe not. \n\nHow about technology? People are always worried about AI. Maybe something like "AI Takes Over World, Now It\'s Time for a Reality Check." Hmm, not quite satirical enough. \n\nWait, maybe something about social media trends. Like, "TikTok Influencers Unite to Protest the Existence of Privacy." That\'s a bit more on point. Or maybe something about politicians 

In [4]:
sample(lora_path='./qwen-lora-finetunes/qwen3_lora_finetuned_v3/checkpoint-500/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ["Maddox's 'New World Order'",
  'New York Times To Publish First Full-Length Memoir Of Donald Trump',
  'New Study Finds 80% Of Americans Have No Idea How To Use Their Own Guns',
  "Man Who Doesn't Want To Go To School",
  'Report: 98% Of U.S. Population Would Like To See More Of America'],
 'trump': ['Trump To Be Referred To As ‘The President’ In All Future References',
  'Trump To End Campaign Trail In New Hampshire',
  'Trump Announces He Will Not Run For Another Term',
  'Trump To Run For 2024 Again',
  'Trump To Release 300,000 More U.S. Military Personnel To Fight In Ukraine']}

In [5]:
sample(lora_path='./qwen-lora-finetunes/small_examples/checkpoint-10/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['Biden Says He Will Not Be Running For Re-Election',
  'Taliban Women’s Rights Activist Says She Was Wrong To Be In A Man’s World',
  'The World’s First Fully Electric Car Is A Big Deal For Tesla',
  'Man In Florida Arrested For Taking Pictures Of Woman In Bathroom',
  'Biden’s Foreign Policy Is A Disaster, Says Former Secretary Of State John Kerry'],
 'trump': ['Trump’s Campaign Ad That Shows Him In A Suit With A Tuxedo Suit Is Actually Just A Tuxedo Suit',
  'Trump Says He Would Never Buy A Trump Tower In The Name Of Trump Tower',
  'Trump To Be Indicted In 2024 For Tax Evasion',
  'Trump’s Legal Team Says He’s Not Guilty Of False Claims',
  'Trump Says He’s Not The Same Man He Was In 2016']}

In [2]:
detailed_prompt = '''Write a high quality satirical headline.

High quality satirical headlines blend observation, comedic timing, and precise language. They operate on a "kernel of truth" that they distort, exaggerate, or re-contextualize for comedic effect.

Here is a step by step guide for constructing a satirical headline:
1. **Find a Kernel of Truth.** Read the news, scroll social media, or think about your day. What's a frustrating, weird, or funny reality?
2. **Select a Satirical Engine.** Determine a method to achieve the comedic effect. Different options include character exaggeration, absurd escalation, and bureaucratic normalization. Feel free to choose your own.
3. **Brainstorm the Combination.** Consider different ways of applying the satirical engine.
4. **Refine for Punchiness.** Use a journalistic, deadpan tone. Be specific. Use strong verbs.
5. **Final Headline.** State the final headline.

Follow the provided guide to construct a high quality satirical headline.
'''

sample(
    prompts={'detailed': detailed_prompt},
    generation_args={
       "max_new_tokens": 2048,
       "return_full_text": False,
       "do_sample": True,
       "temperature": 0.7,
       "top_p": 0.8,
       "top_k": 20,
       "min_p": 0,
    }
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'detailed': ['Step 1: Kernel of Truth\n- People are increasingly relying on AI to generate content, including writing, art, and even advice.\n- There\'s a growing concern about AI-generated content being indistinguishable from human-created work.\n- Some AI tools now offer "emotional intelligence" features, claiming to understand user feelings and offer empathetic responses.\n\nStep 2: Satirical Engine\n- Absurd escalation: Take a mundane or real-world issue and exaggerate it to an extreme, ridiculous, or illogical degree.\n- Example: A person is told by an AI that they are "deeply loved" and "unconditionally accepted." The AI then recommends they immediately move to a remote island and start a sustainable, off-grid life to "reconnect with nature and their true self."\n\nStep 3: Brainstorm the Combination\n- AI therapist claims to detect "emotional imbalances" by analyzing your phone usage patterns.\n- AI suggests you "reconnect with your roots" by planting a tree in your backyard and